Importing some files required for InferSent module

In [ ]:
import os
os.system("mkdir GloVe")
os.system("curl -Lo GloVe/glove.840B.300d.zip http://nlp.stanford.edu/data/glove.840B.300d.zip")
os.system("unzip GloVe/glove.840B.300d.zip -d GloVe/")
os.system("mkdir fastText")
os.system("curl -Lo fastText/crawl-300d-2M.vec.zip https://dl.fbaipublicfiles.com/fasttext/vectors-english/crawl-300d-2M.vec.zip")
os.system("unzip fastText/crawl-300d-2M.vec.zip -d fastText/")
os.system("mkdir encoder")
os.system("curl -Lo encoder/infersent1.pkl https://dl.fbaipublicfiles.com/infersent/infersent1.pkl")
os.system("curl -Lo encoder/infersent2.pkl https://dl.fbaipublicfiles.com/infersent/infersent2.pkl")

In [3]:
#unzipping all files
from zipfile import ZipFile

with ZipFile('../intermediate/lists_to_use.zip', 'r') as f:
    f.extractall('../intermediate/')

#accessing stored contents
import pickle
with open('../intermediate/jgslist_latest.pickle', 'rb') as file:
  jgslist = pickle.load(file)

with open('../intermediate/sumlist_new.pickle', 'rb') as file:
  sumlist = pickle.load(file)

#remove unzipped files
os.system("rm ../intermediate/jgslist_latest.pickle")
os.system("rm ../intermediate/sumlist_new.pickle")

0

In [27]:
import numpy as np
def cosine(u, v):
    return np.dot(u, v) / (np.linalg.norm(u) * np.linalg.norm(v))

In [3]:
! pip install -U pip setuptools wheel
! pip install -U spacy

  Obtaining dependency information for pip from https://files.pythonhosted.org/packages/50/c2/e06851e8cc28dcad7c155f4753da8833ac06a5c704c109313b8d5a62968a/pip-23.2.1-py3-none-any.whl.metadata
  Obtaining dependency information for wheel from https://files.pythonhosted.org/packages/17/11/f139e25018ea2218aeedbedcf85cd0dd8abeed29a38ac1fda7f5a8889382/wheel-0.41.0-py3-none-any.whl.metadata
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 5.2 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.7/64.7 kB 1.4 MB/s eta 0:00:00a 0:00:01
  Attempting uninstall: wheel
    Found existing installation: wheel 0.40.0
    Uninstalling wheel-0.40.0:
      Successfully uninstalled wheel-0.40.0
  Attempting uninstall: pip
    Found existing installation: pip 23.2
    Uninstalling pip-23.2:
      Successfully uninstalled pip-23.2


In [5]:
! python -m spacy validate

✔ Loaded compatibility table

================= Installed pipeline packages (spaCy v3.6.0) =================
ℹ spaCy installation:
/Users/meetbanthia/miniconda3/envs/torch/lib/python3.9/site-packages/spacy

NAME             SPACY            VERSION                            
en_core_web_md   >=3.6.0,<3.7.0   3.6.0   ✔



In [6]:
! python -m spacy download en_core_web_md


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.8/42.8 MB 24.0 MB/s eta 0:00:0000:0100:01
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_md')


You might require to run the below two lines on your terminal:
1. pip install spacy
2. python -m spacy download en_core_web_md

OR

You can create a cell above this and just run both these lines:
1. !pip install spacy
2. !python -m spacy download en_core_web_md


# SpaCy(GloVe)

In [12]:
#used example sentency to just check whether this works fine

#this is glove embeddings
#Spacy uses GLoVE word vectors of 300 dimensions trained on Commoncrawl corpus.

import spacy

nlp = spacy.load('en_core_web_md')
tokens = nlp("apple cat sky")


print(tokens.text, len(tokens.vector), tokens.vector_norm) # Only the first three components of the vector
print(tokens.vector[0])
print(type(tokens.vector))
for token in tokens:
    print(token.text, (token.vector).tolist(), token.vector_norm)

apple cat sky 300 41.221966020414214
3.2824001
<class 'numpy.ndarray'>
apple [-1.0083999633789062, -2.0308001041412354, -0.6418499946594238, 2.6928000450134277, 0.3177100121974945, -2.6661999225616455, -3.7372000217437744, 5.471399784088135, -5.175099849700928, 0.5195800065994263, 5.489299774169922, -2.0917999744415283, -2.4932000637054443, 2.315200090408325, 0.39298999309539795, -2.427299976348877, 3.4245998859405518, -2.6519999504089355, 2.53410005569458, -4.809100151062012, -1.9630999565124512, 4.9045000076293945, -2.543800115585327, -1.5829999446868896, -3.781399965286255, -2.4935998916625977, -2.426100015640259, 3.815700054168701, 0.46178001165390015, 1.228600025177002, -0.20967000722885132, -1.4053000211715698, -2.4086999893188477, -2.226300001144409, 1.4234999418258667, -1.8700000047683716, 2.589200019836426, 3.38319993019104, 2.841599941253662, -1.503100037574768, 1.061400055885315, 1.3466999530792236, 0.7639200091362, 0.9313300251960754, 2.1568000316619873, 2.318000078201294, 

# InferSent

In [55]:
#model creation

# infersent
import torch

#we have model.py file in this directory where we have created our InferSent class
from models import InferSent

V = 2
MODEL_PATH = 'encoder/infersent%s.pkl' % V
params_model = {'bsize': 64, 'word_emb_dim': 300, 'enc_lstm_dim': 2048,
                'pool_type': 'max', 'dpout_model': 0.0, 'version': V}
infersent = InferSent(params_model)
infersent.load_state_dict(torch.load(MODEL_PATH))

#infersent is our model

W2V_PATH = 'fastText/crawl-300d-2M.vec'
infersent.set_w2v_path(W2V_PATH)

In [56]:
#testing our model on example sentences

sentences = ["I ate dinner.",
       "We had a three-course meal.",
       "Brad came to dinner with us.",
       "He loves fish tacos.",
       "In the end, we all felt like we ate too much.",
       "We all agreed; it was a magnificent evening."]

#building vocabulary on our list of sentences
infersent.build_vocab(sentences, tokenize=True)

query = "I had pizza and pasta"
query_vec = infersent.encode([query])[0]
print(type(query_vec))
print(query_vec)

#for infersent

similarity = []
for sent in sentences:
  sim = cosine(query_vec, infersent.encode([sent])[0])
  print("Sentence = ", sent, "; similarity = ", sim)

Found 36(/36) words with w2v vectors
Vocab size : 36
<class 'numpy.ndarray'>
[ 0.00746889 -0.02781818 -0.06682432 ...  0.01568948 -0.06475541
  0.01776477]
Sentence =  I ate dinner. ; similarity =  0.47384942
Sentence =  We had a three-course meal. ; similarity =  0.3102986
Sentence =  Brad came to dinner with us. ; similarity =  0.30405694
Sentence =  He loves fish tacos. ; similarity =  0.09320724
Sentence =  In the end, we all felt like we ate too much. ; similarity =  0.2573072
Sentence =  We all agreed; it was a magnificent evening. ; similarity =  0.30879584


In [ ]:
#for glove sen2vec

similarity = []
for sent in sentences:
  token = nlp(sent)
  sim = cosine(token.vector, query_vec_s2v)
  print("Sentence = ", sent, "; similarity = ", sim)

Sentence =  I ate dinner. ; similarity =  0.7788746
Sentence =  We had a three-course meal. ; similarity =  0.67471653
Sentence =  Brad came to dinner with us. ; similarity =  0.5343362
Sentence =  He loves fish tacos. ; similarity =  0.6137629
Sentence =  In the end, we all felt like we ate too much. ; similarity =  0.6539096
Sentence =  We all agreed; it was a magnificent evening. ; similarity =  0.6515491


# **Using our infersent model and sen2vec in our dataset**

As we have created our InferSent and GloVe model

Now time is to run these models on our train dataset

So idea is to instead of running one among these two models, we will use both of them.

We will generate vectors of our sentence using both our models and then concatenate the vectors to create our final sentence embedding.


There are many words - legal-domain specific words that were not present in the default embeddings.

So we have to do fine tuning to solve this issue or we can use other models(custom pretrained embeddings) or just use other model that performs better like BERT and FastText

Also even stemming and lemmatization can help reduce your number of OOV words.
Stemming and lemmatizing can help reduce the number of out-of-vocabulary words to some extent.

So lets lemmatize first

In [36]:
#lets lemmatize all the words of the dataset
from nltk.stem import WordNetLemmatizer
from nltk import word_tokenize
lemmatizer = WordNetLemmatizer()
jgslist = [[(" ".join([lemmatizer.lemmatize(word) for word in word_tokenize(sent)])) for sent in lst] for lst in jgslist]
sumlist = [[(" ".join([lemmatizer.lemmatize(word) for word in word_tokenize(sent)])) for sent in lst] for lst in sumlist]

In [37]:
#So the first step is to build the vocabulary of InferSent model on our dataset.
#For that we need to paas all our sentences to the model
#so we will first flatten both the jgslist and sumlist into list that contains only sentences, i.e. list of strings
#after that concatenate both the lists into one all_sentences list and pass this to build the vocabulary


# Flatten the jgslist and sumlist
judgement_sentences = [sentence for sublist in jgslist for sentence in sublist]
summary_sentences = [sentence for sublist in sumlist for sentence in sublist]

# Build the vocabulary
# Build the vocabulary
all_sentences = judgement_sentences + summary_sentences
infersent.build_vocab(all_sentences, tokenize=True)

Found 61668(/137664) words with w2v vectors
Vocab size : 61668


Still not good

In [ ]:
#finding sentence embeddings for each sentence

#lst is a list of sentences of a particular judgement
jgsvector = [[(np.concatenate((infersent.encode([sent])[0], (nlp(sent)).vector))).tolist() for sent in lst] for lst in jgslist]

#lst is a list of sentences of a particular judgement
sumvector = [[(np.concatenate((infersent.encode([sent])[0], (nlp(sent)).vector))).tolist() for sent in lst] for lst in sumlist]

In [50]:
# for a particular vector is jgsvector there will be a similarity score
# so for vectors in jgsvector[i] we have to compare each vector with vectors in sumvector[i] and add the scores for that vector in jgsvector[i]
scores = []
for i in range(len(jgsvector)):
  temp = []
  for j in range(len(jgsvector[i])):
    #compare jth sentence with all sentences in sumlist[i] and add scores
    score = 0
    vect1 = np.array(jgsvector[i][j])
    for lst in sumvector[i]:
      vect2 = np.array(lst)
      score += cosine(vect1,vect2)
    temp.append(score)
  scores.append(temp)

In [ ]:
#store my scores list, sentence emebeddings in intermediate folder                      
# Save data to a pickle file
import pickle
with open('../intermediate/scores_inferglo.pickle','wb') as file:
    pickle.dump(scores, file)

with open('../intermediate/jgsvector_inferglo.pickle','wb') as file:
    pickle.dump(jgsvector, file)

with open('../intermediate/sumvector_inferglo.pickle', 'wb') as file:
    pickle.dump(sumvector, file)

import os
os.system("zip ../intermediate/inferglo.zip ../intermediate/scores_inferglo.pickle ../intermediate/jgsvector_inferglo.pickle ../intermediate/sumvector_inferglo.pickle")
os.system("rm ../intermediate/scores_inferglo.pickle")
os.system("rm ../intermediate/jgsvector_inferglo.pickle")
os.system("rm ../intermediate/sumvector_inferglo.pickle")
os.system("rm -r GloVe")
os.system("rm -r fastText")
os.system("rm -r encoder")

# Run the below cell only if you dont want to again use installed models
Thats because if you again want to run this notebook it will save time not installing those folders again